In [61]:
from fenics import * 
import matplotlib.pyplot as plt
import numpy as np
import time

#suppress some output
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)

In [62]:
## Simulation and mesh parameters

# mesh size
Lx = 1.0         
Ly = 1.0

# set grid size
mx = 20
my = int(mx*Ly)

# xi (kpar) slice
J = 21
SHIFT = np.linspace(0.0,1.0,J) - 1.0/3.0

# kperp slice
kpoints = 51;
LAMBDA = np.linspace(-1/2,1/2,kpoints);

# number of eigenvalues
evals = 2*3*int(Lx*Ly)
E = np.zeros((kpoints,J,evals))

In [63]:
## Mesh

# set up honeycomb lattice
q = 4*np.pi/(sqrt(3))
k1 = q*np.array([1,sqrt(3)])/2.0
k2 = q*np.array([1,-sqrt(3)])/2.0
k3 = -(k1+k2)
K = (k1-k2)/3

v1_h = np.array([sqrt(3),1])/2.0
v2_h = np.array([sqrt(3),-1])/2.0

# different edges
# a1 = 1; b1 = 1; a2 = 0; b2 = 1;     # armchair
a1, b1, a2, b2 = 1, 0, 0, 1       # zigzag
# a1 = 3; b1 = 2; a2 = 1; b2 = 1;     # general
# a1 = 2; b1 = 1; a2 = 1; b2 = 1;     # general

v1 = a1*v1_h + b1*v2_h
v2 = a2*v1_h + b2*v2_h

ktilde = -b1*k1+a1*k2         # general

# quasimomentum slice
quasimomentum = np.linspace(0.0,0.5,2)
boundary_0 = K[0]*np.ones((1,2)) + ktilde[0]*quasimomentum
boundary_1 = K[1]*np.ones((1,2)) + ktilde[1]*quasimomentum
boundary = np.concatenate((boundary_0,boundary_1),axis=0)

# create mesh      
def honeycomb_cylinder(x, y):
    return [v1[0]*x + v2[0]*y, v1[1]*x + v2[1]*y]

mesh = RectangleMesh(Point(-Lx/2,-Ly/2),Point(Lx/2,Ly/2),mx,my)#, 'left') # “left”, “right”, “left/right”, “crossed”
x = mesh.coordinates()[:,0]
y = mesh.coordinates()[:,1]

x_hat, y_hat = honeycomb_cylinder(x, y)
xy_hat_coor = np.array([x_hat, y_hat]).transpose()
mesh.coordinates()[:] = xy_hat_coor
#plot(mesh), interactive()

In [64]:
## Boundary Conditions

# sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv
        return bool( ( near(y_r, -Ly/2.0) or near(y_r, Ly/2.0) ) and on_boundary)
    
# sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        return bool( ( near(x_r, -Lx/2.0) and near(x_r, -Lx/2.0) ) and on_boundary)

    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        x_h = x_r - (Lx - 0)
        y_h = y_r
        
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h
        
## perfectly periodic problem

# sub domain for Periodic boundary condition
class PerfectlyPeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        return bool( (near(x_r, -Lx/2.0) or near(y_r, -Ly/2.0)) and 
                    (not ((near(x_r, -Lx/2.0) and near(y_r, +Ly/2.0)) or 
                            (near(x_r, +Lx/2.0) and near(y_r, -Ly/2.0))) ) and on_boundary)
        
    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        if near(x_r, +Lx/2.0) and near(y_r, +Ly/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r - Ly - 0.0
        elif near(x_r, +Lx/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r
        else:   # near(y_r, +Ly/2.0)
            x_h = x_r 
            y_h = y_r - Ly - 0.0
            
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h

In [65]:
## Finite Elements

# periodic = PeriodicBoundary()
periodic = PerfectlyPeriodicBoundary()

P2 = FiniteElement("P", triangle, 2)
element = MixedElement([P2, P2])
W = FunctionSpace(mesh, element, constrained_domain=periodic)

# define functions
(u_r, u_i) = TrialFunctions(W)
(v_r, v_i) = TestFunctions(W)

In [66]:
## Potential terms
V_e = Expression('4.0-1.0*(cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1]))', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
W_o = Expression('sin(k10*x[0]+k11*x[1]) + sin(k20*x[0]+k21*x[1]) + sin(k30*x[0]+k31*x[1])', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
V_e_2 = Expression('cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1])', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)

delta = 1.0

C_r = as_matrix( ((V_e, 0.0), (0.0, V_e)) )
C_i = as_matrix( ((0.0, -delta*V_e_2), (delta*V_e_2, 0.0)) )

In [67]:
## Computations

# assemble mass matrix
m_r = (u_r*v_r + u_i*v_i)*dx
m_i = (-u_r*v_i + u_i*v_r)*dx
M = PETScMatrix()
assemble(m_r+m_i, tensor=M)

#L = inner(Constant(1.0), v_r+v_i)*dx
#b = PETScVector()
#assemble_system(m_r+m_i, L, A_tensor=M, b_tensor=b)

# eigenvalue solver
def compute_evals(A, E, M, l, r):
    
    # create eigensolver
    eigensolver = SLEPcEigenSolver(A,M)
    eigensolver.parameters['spectrum'] = 'smallest real'
    eigensolver.parameters['solver']   = 'krylov-schur'
    eigensolver.parameters['tolerance'] = 1.e-8   # 1.e-10
    #eigensolver.parameters['maximum_iterations'] = 1000
    #eigensolver.parameters['problem_type'] = 'gen_hermitian'


    # solve for eigenvalues
    eigensolver.solve(evals)
    
    for i in range(0,evals):
        if i <= (eigensolver.get_number_converged()-1):
            E[l][r][i] = eigensolver.get_eigenvalue(i)[0]
    
    #return np.sort(E)
    return E

program_starts = time.time()

r = 0

# main computation
for xi in SHIFT:
    
    k_vec = K+xi*k1
    
    l = 0    
    for lam in LAMBDA:
    
        kk = k_vec + lam*ktilde
        k = Constant(kk)

        # define problem 
        a_r = ( inner(C_r*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_r*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) +\
               -inner(C_i*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) + inner(C_i*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) )*dx
        a_i = ( inner(C_r*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) - inner(C_r*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) +\
               +inner(C_i*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_i*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) )*dx

        # assemble stiffness matrix
        A = PETScMatrix()
        assemble(a_r+a_i, tensor=A)

        # assemble_system(a_r+a_i, L, bcs, A_tensor=A, b_tensor=b)

        E = compute_evals(A, E, M, l, r)
        
        l += 1

    r += 1
    print r/float(J)
    
print time.time() - program_starts

0.047619047619
0.0952380952381
0.142857142857
0.190476190476
0.238095238095
0.285714285714
0.333333333333
0.380952380952
0.428571428571
0.47619047619
0.52380952381
0.571428571429
0.619047619048
0.666666666667
0.714285714286
0.761904761905
0.809523809524
0.857142857143
0.904761904762
0.952380952381
1.0
1904.9841311


In [68]:
for x in range(0,3,2):
    Emin = [1e4]*J
    Emax = [0]*J
    for Eslice in E:
        for j in range(len(Eslice)):
            #newMin = min([e for e in Eslice[j] if e>0])
            if Eslice[j][x] == 0.0:
                continue
            if Eslice[j][x] < Emin[j]:
                Emin[j] = Eslice[j][x]
            if Eslice[j][x] > Emax[j]:
                Emax[j] = Eslice[j][x]
    print 'ymin =', Emin, ';'
    print ''
    print 'ymax =', Emax, ';'
    print ''
  
# MATLAB plotting
# y = [ymin,ymax,ymin(1)]
# x = [linspace(0,2*pi,31),linspace(2*pi,0,31),0]
# figure; fill(x,y,'k')
# hold on; fill(x,y,'k')

ymin = [0.010829922232326827, 0.34195471951886702, 1.3577447353264587, 3.0524541687469786, 5.3918124377619758, 8.3715424609815834, 11.977492593490966, 16.167959016701577, 20.851098418969809, 25.914991068274944, 31.129893707004729, 25.932999145728648, 20.857183356193595, 16.162265574417209, 11.98292382949127, 8.388716400503883, 5.3972849819554574, 3.0460208903660431, 1.3633487753228328, 0.35964949276591224, 0.016462218904234594] ;

ymax = [35.287020964500016, 35.557256585727657, 36.184586645734385, 37.137688880793227, 38.480064227244533, 39.947058067862336, 41.114846989368871, 41.37681703445449, 39.717330039985491, 36.553798400892504, 35.362462918323651, 36.562728199514844, 39.686299544695459, 41.339386052660544, 41.119870468239924, 39.897890805657681, 38.48787776469797, 37.219057543266885, 36.191457248142427, 35.479837722942783, 35.295251908832334] ;

ymin = [55.20234879439991, 55.543632176374558, 56.770477381430247, 58.866830920432093, 61.656401861295848, 65.228612917386343, 68.183747

In [69]:
E

array([[[  19.38066617,   19.38066617,   77.58885491,  140.01295466,
          149.65635051,  232.48891693],
        [  22.50145337,   22.50145337,   73.28042797,  130.41123955,
          156.89115299,  236.25303134],
        [  26.27808878,   26.27808878,   69.69335513,  118.2298312 ,
          168.34684906,  237.00222416],
        ..., 
        [  15.46658745,   15.46658745,   86.44675318,  127.05118455,
            0.        ,    0.        ],
        [  17.01757539,   17.0175754 ,   82.13792287,   82.13792288,
          134.03915317,  159.2265978 ],
        [  19.38074082,   19.38074083,   77.49758116,   77.49758117,
          139.73336794,  149.38607001]],

       [[  17.22337543,   17.22337543,   81.25469375,   81.25469377,
          140.99582805,  140.99582805],
        [  20.24256326,   20.24256327,   76.82753029,  131.65871061,
          158.04201544,  231.33645031],
        [  23.97314744,   23.97314744,   72.99357853,   72.99357855,
          119.51443359,  169.15170267],
   